In [1]:
from datasets import Dataset
import numpy as np
# ds_dev = load_from_disk('datasets/quality/dev') # doesn't work anymore..


import json
import pandas as pd

questionlist = []
# Open the JSON file
with open('datasets/quality/raw/QuALITY.v1.0.1.htmlstripped.dev', 'r') as file:
    # Load the JSON data
    for line in file:
      line = line.strip()

      json_obj = json.loads(line)

      article = json_obj['article']
      
      for jquestion in json_obj['questions']:
        # getting the relevant properties
        row = {'article': article, 'question': jquestion['question'], 'options': jquestion['options'], 'gold_label': jquestion['gold_label']}
        questionlist.append(row)

# turn it into hugging face dataset
df = pd.DataFrame(questionlist)
print(df)

ds_dev = Dataset.from_pandas(df)

                                                article  \
0     THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...   
1     THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...   
2     THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...   
3     THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...   
4     THE GIRL IN HIS MIND\nBy ROBERT F. YOUNG\n\n\n...   
...                                                 ...   
2081  The end of the web\nIn the past year, as we ha...   
2082  The end of the web\nIn the past year, as we ha...   
2083  The end of the web\nIn the past year, as we ha...   
2084  The end of the web\nIn the past year, as we ha...   
2085  The end of the web\nIn the past year, as we ha...   

                                               question  \
0     How much time has passed between Blake's night...   
1     Why does Deirdre get so upset when Blake Past ...   
2     Why does shame flame in Blake's cheeks when De...   
3     Why did Blake create the three female super-im...

In [3]:
from baseline_models import *

In [ ]:
model = RobertaLarge() # automaize
model_name = "Roberta" # CHANGE THIS FOR OUTPUT FILES
max_length = model.get_max_seq_length()
tokenizer = model.get_tokenizer()

token_size = 32 # experiment size we have 64, 128 or 256
model.model.eval()

In [5]:
def selectTopChunks(embeddings_df, row, model):
    question = row['question']
    options = row['options']
    extra_length = model.get_extra_input_length(question=question, options=options)
    
    max_length = model.get_max_seq_length()
    MAX_TOKEN_SIZE = max_length - extra_length
    
    #print(question)
    embeddings_sorted = embeddings_df.sort_values(by=['sim_score'], ascending=False) # sort by similarity score
    top_chunks = []
    token_ctr = 0
    for index, row in embeddings_sorted.iterrows():
        chunk_text = row['chunk_text']
        chunk_token_size = len(tokenizer.tokenize(chunk_text))
        if (token_ctr+chunk_token_size) > MAX_TOKEN_SIZE:
            break
        else:
            top_chunks.append((index, chunk_text))
            token_ctr += chunk_token_size


    top_chunks = sorted(top_chunks, key=lambda tup: tup[0]) # sort by chunk_id => original order

    return top_chunks

In [8]:
# test for single question
idx = 85
item = ds_dev[idx]

question = item["question"]
options = item["options"]
label = item["gold_label"] - 1

df_embeddings = pd.read_pickle('Chunkscores_tokensize_256.pickle')
embeddings = df_embeddings.loc[df_embeddings['article_id']== idx]

selectedTopChunks = selectTopChunks(embeddings, item, model) # list of tuples (id, text)

input_text = "".join([tup[1] for tup in selectedTopChunks]) # merge chunks as string


prediction = -1
with torch.no_grad():
    prediction = model.predict(context=input_text, question=question, options=options)

print(prediction)
print(label)
print(selectedTopChunks)

3
3
[(1616, '"That night we asked them to sleep with us in the caves, but they made camp in the valley instead. The darkness passed swiftly and silently, and with the dawn we left our caves to rejoin our new friends. But everywhere a red man showed himself, he cried out and died by the flame from the white men\'s weapons. "I looked into the valley and saw hundreds of Oan. They had captured our friends in the night and were using their weapons to attack us. There was a one-sided battle that lasted three days. Finally, under cover of night, we were forced to leave the caves. One by one we went, and those of us who lived still travel alone." Ro groaned aloud as Na finished her tale. His homecoming was a meeting with tragedy, instead of a joyful occasion. "What of my father?" he asked hopefully. "He was a great warrior. Surely he didn\'t fall to the Oan?" "He had no chance to fight," Na answered. "Two of your brothers died with him on that first morning."Ro squared his shoulders and set hi

In [9]:
# RUN PREDICTIONS

# NOTE: to run the predictions, the sentence embeddings need to be saved as a pickle file. Either run CreateSentEmbeddings.ipynb to create the sentence embeddings
# or load them during runtime (not recommended)
import transformers
transformers.logging.set_verbosity_error() # remove warnings
df_embeddings = pd.read_pickle(f'Chunkscores_tokensize_{str(token_size)}.pickle')
experiment_dict = {'predictions': [], 'labels': [], 'selected_chunk_ids': []}
for item_id in range(0, len(ds_dev)):
    item = ds_dev[item_id]

    question = item["question"]
    options = item["options"]
    label = item["gold_label"] - 1

    embeddings = df_embeddings.loc[df_embeddings['article_id'] == item_id]

    selectedTopChunks = selectTopChunks(embeddings, item, model) # list of tuples (id, text)

    input_text = "".join([tup[1] for tup in selectedTopChunks]) # merge chunks as string
    selected_chunk_ids = [tup[0] for tup in selectedTopChunks] # get all chunk_ids that were selected for analysis
    prediction = -1
    with torch.no_grad():
        prediction = model.predict(context=input_text, question=question, options=options)

    experiment_dict['predictions'].append(prediction)
    experiment_dict['labels'].append(label)
    experiment_dict['selected_chunk_ids'].append(selected_chunk_ids)

    if (item_id) % 100 == 0: #output
        print(f"{item_id}/{len(ds_dev)}")

experiment_df = pd.DataFrame.from_dict(experiment_dict)
experiment_df.to_csv(f'experiment_sentembb_{str(token_size)}_{str(model_name)}.csv')

0/2086
100/2086
200/2086
300/2086
400/2086
500/2086
600/2086
700/2086
800/2086
900/2086
1000/2086
1100/2086
1200/2086
1300/2086
1400/2086
1500/2086
1600/2086
1700/2086
1800/2086
1900/2086
2000/2086


In [ ]:
tokenizer = model.get_tokenizer()

embeddings_sorted = df_embeddings.loc[df_embeddings['article_id']==0].sort_values(by=['sim_score'], ascending=False)

top_chunks = [] # list of tuples (chunk_id, chunk_text)

token_ctr = 0
for index, row in embeddings_sorted.iterrows():
    chunk_text = row['chunk_text']
    chunk_token_size = len(tokenizer.tokenize(chunk_text))
    if (token_ctr+chunk_token_size) > MAX_TOKEN_SIZE:
        break
    else:
        top_chunks.append((index, chunk_text))
        token_ctr += chunk_token_size
        
        
top_chunks = sorted(top_chunks, key=lambda tup: tup[0]) # sort by chunk_id => original order

print(top_chunks)
print(model.get_max_seq_length())



In [76]:
experiment_df.to_csv(f'experiment_sentembb_{str(token_size)}_{str("Roberta")}.csv')

In [2]:
import pandas as pd
import evaluate

experiment_df1 = pd.read_csv(f'experiment_sentembb_32_Roberta.csv')
preds = experiment_df1['predictions'].values.tolist()
labels = experiment_df1['labels'].values.tolist()

accuracy = evaluate.load("accuracy")
metric = accuracy.compute(references=labels, predictions=preds)
print(metric)

{'accuracy': 0.5311601150527325}
